In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
pip install -i https://test.pypi.org/simple/ my_krml_25076922==2024.0.3.4

Looking in indexes: https://test.pypi.org/simple/
Note: you may need to restart the kernel to use updated packages.


In [3]:
import pandas as pd
import numpy as np

/Users/vega7unk/anaconda3/lib/python3.11/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [4]:
path ='/Users/vega7unk/Documents/3rd_Sem_DSI/Adv_ML/AT2/Project/adv_mla_at2/data/processed'

sales_data = pd.read_csv(f'{path}/df_pred.csv')

In [5]:
sales_data.head()

,item_id,store_id,date,total_sales,day_of_week,month,year
0,HOBBIES_1_001,CA_1,2011-01-29,0.0,5,1,2011
1,HOBBIES_1_002,CA_1,2011-01-29,0.0,5,1,2011
2,HOBBIES_1_003,CA_1,2011-01-29,0.0,5,1,2011
3,HOBBIES_1_004,CA_1,2011-01-29,0.0,5,1,2011
4,HOBBIES_1_005,CA_1,2011-01-29,0.0,5,1,2011


In [6]:
from my_krml_25076922.features.data_processing import reduce_mem_usage

# Apply the memory reduction function to the sales_train_long DataFrame
sales_data = reduce_mem_usage(sales_data, int_cast=True, obj_to_category=True)

Memory usage of dataframe is 2515.79 MB


100%|██████████████████████████████| 7/7 [00:04<00:00,  1.51it/s]

Memory usage after optimization is: 494.303 MB
Decreased by 80.4%


In [7]:
sales_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 47107050 entries, 0 to 47107049
Data columns (total 7 columns):
 #   Column       Dtype   
---  ------       -----   
 0   item_id      category
 1   store_id     category
 2   date         category
 3   total_sales  float16 
 4   day_of_week  int8    
 5   month        int8    
 6   year         int16   
dtypes: category(3), float16(1), int16(1), int8(2)
memory usage: 494.3 MB


In [8]:
X = sales_data.drop(columns=['total_sales','date'])  # Features
y = sales_data['total_sales']  # Target

In [9]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)


In [10]:
print(X_train.shape)
print(X_val.shape)
print(X_test.shape)

(30148512, 5)
(7537128, 5)
(9421410, 5)


In [11]:
print(y_train.shape)
print(y_val.shape)
print(y_test.shape)

(30148512,)
(7537128,)
(9421410,)


In [12]:
categorical_features = X.select_dtypes(include=['category', 'object']).columns

# Convert categorical columns to 'category' dtype
categorical_features = X.select_dtypes(include=['category', 'object']).columns
for cat_feature in categorical_features:
    X_train[cat_feature] = X_train[cat_feature].astype('category')
    X_val[cat_feature] = X_val[cat_feature].astype('category')
    X_test[cat_feature] = X_test[cat_feature].astype('category')

In [13]:
import lightgbm as lgb

train_data = lgb.Dataset(X_train, label=y_train, categorical_feature=categorical_features.tolist())
val_data = lgb.Dataset(X_val, label=y_val, categorical_feature=categorical_features.tolist(), reference=train_data)


In [14]:
# Parameters for LightGBM
params = {
    'objective': 'regression',
    'metric': 'rmse',
    'boosting_type': 'gbdt',
    'num_leaves': 31,
    'learning_rate': 0.05,
    'feature_fraction': 0.9,
    'random_state': 42
}

# Train the LightGBM model
lgb_model = lgb.train(
    params,
    train_data,
    valid_sets=[train_data, val_data],
    num_boost_round=1000,
    early_stopping_rounds=100,
    verbose_eval=100
)

/Users/vega7unk/anaconda3/lib/python3.11/site-packages/lightgbm/engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/Users/vega7unk/anaconda3/lib/python3.11/site-packages/lightgbm/basic.py:2065: UserWarning: Using categorical_feature in Dataset.
  _log_warning('Using categorical_feature in Dataset.')
/Users/vega7unk/anaconda3/lib/python3.11/site-packages/lightgbm/engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "


[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.100591 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 3044
[LightGBM] [Info] Number of data points in the train set: 30148512, number of used features: 5


/Users/vega7unk/anaconda3/lib/python3.11/site-packages/lightgbm/basic.py:1780: UserWarning: Overriding the parameters from Reference Dataset.
  _log_warning('Overriding the parameters from Reference Dataset.')
/Users/vega7unk/anaconda3/lib/python3.11/site-packages/lightgbm/basic.py:1513: UserWarning: categorical_column in param dict is overridden.
  _log_warning(f'{cat_alias} in param dict is overridden.')


[LightGBM] [Info] Start training from score 3.031970
Training until validation scores don't improve for 100 rounds
[100]	training's rmse: 7.03111	valid_1's rmse: 7.04569
[200]	training's rmse: 6.74219	valid_1's rmse: 6.75644
[300]	training's rmse: 6.62261	valid_1's rmse: 6.63914
[400]	training's rmse: 6.54506	valid_1's rmse: 6.56268
[500]	training's rmse: 6.49786	valid_1's rmse: 6.51787
[600]	training's rmse: 6.46318	valid_1's rmse: 6.48401
[700]	training's rmse: 6.42791	valid_1's rmse: 6.45005
[800]	training's rmse: 6.39704	valid_1's rmse: 6.42123
[900]	training's rmse: 6.36853	valid_1's rmse: 6.3948
[1000]	training's rmse: 6.34267	valid_1's rmse: 6.36939
Did not meet early stopping. Best iteration is:
[1000]	training's rmse: 6.34267	valid_1's rmse: 6.36939


In [15]:
y_val_pred = lgb_model.predict(X_val, num_iteration=lgb_model.best_iteration)

In [17]:
from sklearn.metrics import mean_squared_error, mean_absolute_error

#Evaluation metrics on validation set
rmse_val = np.sqrt(mean_squared_error(y_val, y_val_pred))
mae_val = mean_absolute_error(y_val, y_val_pred)

print(f'Validation RMSE: {rmse_val:.4f}')
print(f'Validation MAE: {mae_val:.4f}')

Validation RMSE: 6.3694
Validation MAE: 3.0269


In [18]:
y_test_pred = lgb_model.predict(X_test, num_iteration=lgb_model.best_iteration)

# Evaluation metrics on test set
rmse_test = np.sqrt(mean_squared_error(y_test, y_test_pred))
mae_test = mean_absolute_error(y_test, y_test_pred)

print(f'Test RMSE: {rmse_test:.4f}')
print(f'Test MAE: {mae_test:.4f}')

Test RMSE: 6.3937
Test MAE: 3.0248


In [19]:
import joblib

joblib.dump(lgb_model, '/Users/vega7unk/Documents/3rd_Sem_DSI/Adv_ML/AT2/Project/adv_mla_at2/models/predictive/lightgbm_sales_model.pkl')
print("Model saved as 'lightgbm_sales_model.pkl'")

Model saved as 'lightgbm_sales_model.pkl'
